In [26]:
#importing useful packages
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.io import ascii
import astropy.time
from astropy.time import Time
import astropy.table
from astropy.table import Table, Column, MaskedColumn
from astroplan import Observer
from astroquery.jplhorizons import Horizons
import pandas as pd
import csv
import astropy.time
import dateutil.parser

### Reading CSV file of dates targets were observed
raw_target_info = pd.read_csv("input_obs_data.csv")

In [27]:
### Creating new dataframe with first date each target was observed
subset_df = raw_target_info[["Target","Date"]]
grouped_and_first_df = subset_df.groupby(["Target"]).first()
grouped_and_first_df["N"] = subset_df.groupby(["Target"]).nunique()
grouped_and_first_df

new_df = grouped_and_first_df.reset_index()
new_df.columns = ["Name","first_date","N"]
new_df.head(2)

# there is something wrong with Astropy's implementation of JPL Horizons (some kind of unit conversion error), which is why I have manually looked up the values of 'q' and added them as a list here.
q_au_df = pd.DataFrame([0.8474057675311619,0.6634838688106598,0.9742705773237288,0.9853271890919407,0.5753015667052206,0.9682696724261184,1.015084825837723,0.9922111606832896,0.9202713846766689,0.7013086053529216,10.95318284865856,3.957327001114813,3.439958406444498,10.31606795740346,2.95714245711804,1.412974951279192,7.928967772429115,1.96767991935478,2.160159379572061,3.241629017720665,4.976803191778574,1.318137370154377,1.735028260450141,6.230275849969386,5.148568866680009],columns=['q'])

In [28]:
### 
target_location = '474' # Mt. John Observatory MPC code

obsDate_lst = []
for i in new_df["first_date"]:
    obsDate_lst.append(i)

targetname_lst = []
for i in new_df.Name:
    targetname_lst.append(i)

# need to convert epoch date to julian date
jd_obsDate_lst = []
for i in obsDate_lst:
    dt = dateutil.parser.parse(obsDate_lst[0]) 
    time = astropy.time.Time(dt)
    jd_obsDate_lst.append(time.jd)

In [29]:
# #need to query information from JPL Horizons database
eph_lst = []
el_lst = []
obj_lst = []
for index,jdDate in enumerate(jd_obsDate_lst):
    if targetname_lst[index] == "C/2018 F4":
        obj = Horizons(id='90004395', epochs=jdDate)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)
    else:
        obj = Horizons(id=targetname_lst[index], epochs=jdDate)
        obj_lst.append(obj)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)


# to check elements returned by obj.elements, see:
# https://astroquery.readthedocs.io/en/latest/api/astroquery.jplhorizons.HorizonsClass.html#astroquery.jplhorizons.HorizonsClass.elements

In [30]:
final_array = []
for index,target in enumerate(eph_lst):
    final_array.append([el_lst[index]['a'][0],eph_lst[index]['r'][0],
                        el_lst[index]['Tp_jd'][0]])

In [31]:
eph_el_df = pd.DataFrame(final_array,columns=['a','r','T'])
eph_el_df.head(5)

,a,r,T
0,-0.000013,1.023791,2.459241e+06
1,-0.000016,1.104592,2.459245e+06
2,-0.000490,0.983340,2.459180e+06
3,-0.000141,1.022337,2.459254e+06
4,-0.000010,1.139992,2.459260e+06


In [32]:
final_new_df = pd.concat([new_df,eph_el_df,q_au_df],axis=1)
final_new_df.head(5)

,Name,first_date,N,a,r,T,q
0,189040,21/01/2021,1,-0.000013,1.023791,2.459241e+06,0.847406
1,2009 CD,11/02/2021,3,-0.000016,1.104592,2.459245e+06,0.663484
2,2020 UB5,11/02/2021,4,-0.000490,0.983340,2.459180e+06,0.974271
3,2021 CA1,11/02/2021,3,-0.000141,1.022337,2.459254e+06,0.985327
4,2021 CD1,12/02/2021,3,-0.000010,1.139992,2.459260e+06,0.575302


In [33]:
final_new_df = final_new_df[['Name','first_date','a','N','r','q','T']]
final_new_df.head(5)

,Name,first_date,a,N,r,q,T
0,189040,21/01/2021,-0.000013,1,1.023791,0.847406,2.459241e+06
1,2009 CD,11/02/2021,-0.000016,3,1.104592,0.663484,2.459245e+06
2,2020 UB5,11/02/2021,-0.000490,4,0.983340,0.974271,2.459180e+06
3,2021 CA1,11/02/2021,-0.000141,3,1.022337,0.985327,2.459254e+06
4,2021 CD1,12/02/2021,-0.000010,3,1.139992,0.575302,2.459260e+06


In [34]:
final_new_df['T'] = final_new_df['T'].map(lambda name: Time(name, format='jd').to_value('iso')[:10])
final_new_df.head(5)

,Name,first_date,a,N,r,q,T
0,189040,21/01/2021,-0.000013,1,1.023791,0.847406,2021-01-26
1,2009 CD,11/02/2021,-0.000016,3,1.104592,0.663484,2021-01-30
2,2020 UB5,11/02/2021,-0.000490,4,0.983340,0.974271,2020-11-26
3,2021 CA1,11/02/2021,-0.000141,3,1.022337,0.985327,2021-02-08
4,2021 CD1,12/02/2021,-0.000010,3,1.139992,0.575302,2021-02-14


In [35]:
final_new_df['a'] = np.round(final_new_df['a'], decimals = 6)
final_new_df['r'] = np.round(final_new_df['r'], decimals = 2)
#final_new_df['q'] = final_new_df['q']  * (6.6846 * 10**-9)
final_new_df['q'] = np.round(final_new_df['q'], decimals = 2)
final_new_df.head(2)

,Name,first_date,a,N,r,q,T
0,189040,21/01/2021,-0.000013,1,1.02,0.85,2021-01-26
1,2009 CD,11/02/2021,-0.000016,3,1.10,0.66,2021-01-30


In [36]:
final_new_df = final_new_df.drop(columns="first_date")

final_column_names = ['Name','$a_0$ [au]','N','$r_0$ [au]','$q$ [au]','T']
final_new_df.columns = final_column_names
final_new_df.head(2)

,Name,$a_0$ [au],N,$r_0$ [au],$q$ [au],T
0,189040,-0.000013,1,1.02,0.85,2021-01-26
1,2009 CD,-0.000016,3,1.10,0.66,2021-01-30


In [37]:
final_new_df.to_csv("final_look_table.csv")
final_new_df.to_latex(columns=final_column_names)

'\\begin{tabular}{llrrrrl}\n\\toprule\n{} &          Name &  \\$a\\_0\\$ [au] &  N &  \\$r\\_0\\$ [au] &  \\$q\\$ [au] &           T \\\\\n\\midrule\n0  &        189040 &   -0.000013 &  1 &        1.02 &      0.85 &  2021-01-26 \\\\\n1  &       2009 CD &   -0.000016 &  3 &        1.10 &      0.66 &  2021-01-30 \\\\\n2  &      2020 UB5 &   -0.000490 &  4 &        0.98 &      0.97 &  2020-11-26 \\\\\n3  &      2021 CA1 &   -0.000141 &  3 &        1.02 &      0.99 &  2021-02-08 \\\\\n4  &      2021 CD1 &   -0.000010 &  3 &        1.14 &      0.58 &  2021-02-14 \\\\\n5  &       2021 CH &   -0.000035 &  4 &        1.08 &      0.97 &  2021-02-10 \\\\\n6  &       2021 CM &   -0.000058 &  5 &        1.02 &      1.02 &  2021-02-06 \\\\\n7  &       2021 CP &   -0.000033 &  3 &        0.99 &      0.99 &  2021-02-03 \\\\\n8  &       2021 CS &   -0.000015 &  2 &        1.03 &      0.92 &  2021-02-01 \\\\\n9  &        360502 &   -0.000032 &  1 &        1.05 &      0.70 &  2021-01-16 \\\\\n10 &  C/20

In [38]:
#WRITING TABLES!!!
final_look_table = Table.from_pandas(final_new_df)
ascii.write(final_look_table,format="latex",
            formats={'$a_0$ [au]':'%12.6f'})

\begin{table}
\begin{tabular}{cccccc}
Name & $a_0$ [au] & N & $r_0$ [au] & $q$ [au] & T \\
189040 & -0.000013 & 1 & 1.02 & 0.85 & 2021-01-26 \\
2009 CD & -0.000016 & 3 & 1.1 & 0.66 & 2021-01-30 \\
2020 UB5 & -0.000490 & 4 & 0.98 & 0.97 & 2020-11-26 \\
2021 CA1 & -0.000141 & 3 & 1.02 & 0.99 & 2021-02-08 \\
2021 CD1 & -0.000010 & 3 & 1.14 & 0.58 & 2021-02-14 \\
2021 CH & -0.000035 & 4 & 1.08 & 0.97 & 2021-02-10 \\
2021 CM & -0.000058 & 5 & 1.02 & 1.02 & 2021-02-06 \\
2021 CP & -0.000033 & 3 & 0.99 & 0.99 & 2021-02-03 \\
2021 CS & -0.000015 & 2 & 1.03 & 0.92 & 2021-02-01 \\
360502 & -0.000032 & 1 & 1.05 & 0.7 & 2021-01-16 \\
C/2014 UN271 & -0.000005 & 5 & 20.74 & 10.95 & 2019-09-01 \\
C/2017 W2 & -0.000002 & 2 & 8.4 & 3.96 & 2020-08-14 \\
C/2018 F4 & -0.000002 & 3 & 10.85 & 3.44 & 2020-12-26 \\
C/2019 E3 & -0.000002 & 4 & 11.66 & 10.32 & 2021-08-11 \\
C/2020 R7 & -0.000005 & 3 & 6.24 & 2.96 & 2020-11-02 \\
C/2021 A2 & -0.000001 & 1 & 1.41 & 1.41 & 2021-02-04 \\
C/2021 A6 & -0.000003 & 3 &